In [7]:
import pandas as pd
import numpy as np
import skopt


In [59]:
from sklearn.metrics import accuracy_score

In [53]:
from sklearn.linear_model import LinearRegression

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from skopt import BayesSearchCV

In [9]:
from sklearn.svm import SVC

In [10]:
train=pd.read_csv('train.csv')

In [11]:
train.shape

(891, 12)

In [12]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [14]:
train.drop('Cabin',axis=1,inplace=True)


In [15]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [16]:
import seaborn as sns

In [17]:
test=pd.read_csv('test.csv')



In [18]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [19]:
train_test_df = [test, train]

for data in train_test_df:
    data['Tittle'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [20]:
# fill missing age with median age for each title (Mr, Mrs, Miss, Others)
train["Age"].fillna(train.groupby("Tittle")["Age"].transform("median"), inplace=True)
test["Age"].fillna(test.groupby("Tittle")["Age"].transform("median"), inplace=True)

In [21]:
test.drop('Cabin',axis=1,inplace=True)

In [22]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Tittle
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,Mrs


In [23]:
train.drop(['Name', 'Ticket', 'Tittle'], axis =1, inplace=True)

In [24]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [25]:
 train['Sex'] = train['Sex'].map( {'female': 1, 'male': 0} ).astype(int)



In [26]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,S
1,2,1,1,1,38.0,1,0,71.2833,C
2,3,1,3,1,26.0,0,0,7.9250,S
3,4,1,1,1,35.0,1,0,53.1000,S
4,5,0,3,0,35.0,0,0,8.0500,S


In [27]:
 test['Sex'] = test['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [28]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Tittle
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,Q,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,S,Mrs
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,Q,Mr
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,S,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,S,Mrs


In [29]:
test.drop(['Name', 'Ticket', 'Tittle'], axis =1, inplace=True)

In [30]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
dtype: int64

In [31]:
train['Embarked'].fillna('S',inplace=True)
train['Embarked'] = train['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
# test['Embarked'] = test['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)


In [32]:
test['Embarked'] = test['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,0
1,2,1,1,1,38.0,1,0,71.2833,1
2,3,1,3,1,26.0,0,0,7.9250,0
3,4,1,1,1,35.0,1,0,53.1000,0
4,5,0,3,0,35.0,0,0,8.0500,0


In [33]:
test.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            1
SibSp          0
Parch          0
Fare           1
Embarked       0
dtype: int64

In [34]:
test.fillna(method='ffill',inplace=True)

In [35]:
train.drop('PassengerId',axis=1,inplace=True)

In [36]:
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [37]:
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

In [38]:
search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1)

NameError: name 'BayesSearchCV' is not defined

In [39]:
search.fit(X_train, Y_train)

NameError: name 'search' is not defined

In [40]:
print(search.best_score_)

NameError: name 'search' is not defined

In [ ]:

print(search.best_params_)

In [47]:



from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)


de_test_pred = decision_tree.predict(X_test)
de_train_pred = decision_tree.predict(X_train)



In [48]:
random = RandomForestClassifier()
random.fit(X_train,Y_train)


ran_test_pred = random.predict(X_test)
ran_train_pred = random.predict(X_train)


In [49]:
svc = SVC()
svc.fit(X_train,Y_train)


svc_test_pred = svc.predict(X_test)
svc_train_pred = svc.predict(X_train)

In [55]:
first_level_train = pd.DataFrame(svc_train_pred, columns=['svc'])
first_level_train['random'] = ran_train_pred
first_level_train['decision'] = de_train_pred
first_level_train.head(10)

,svc,random,decision
0,0,0,0
1,1,1,1
2,0,1,1
3,1,1,1
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,0,1,1
9,0,1,1


In [50]:
first_level = pd.DataFrame(svc_test_pred, columns=['svc'])
first_level['random'] = ran_test_pred
first_level['decision'] = de_test_pred
first_level.head(10)

,svc,random,decision
0,0,0,0
1,0,0,0
2,0,0,1
3,0,1,1
4,0,0,1
5,0,0,0
6,0,0,0
7,0,0,0
8,0,1,1
9,0,0,0


In [54]:
meta_model = LinearRegression(n_jobs=-1)

In [57]:
#first_level_train.drop('label', axis=1, inplace=True)
meta_model.fit(first_level_train, Y_train)

LinearRegression(n_jobs=-1)

In [58]:
ensemble_pred = meta_model.predict(first_level_train)
final_predictions = meta_model.predict(first_level)

In [62]:
#acc = accuracy_score(Y_test,ensemble_pred)

In [81]:
final_predictions = pd.DataFrame(final_predictions, columns=['survived'])

In [ ]:
fina;_predictions['survived'] 

In [78]:
output = pd.DataFrame({
                    'PassengerId': test.PassengerId,
                    'Survived' :final_predictions
})

output.to_csv('submission14.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [79]:
output.head()

,PassengerId,Survived
0,892,0.020638
1,893,0.020638
2,894,0.514232
3,895,0.993594
4,896,0.514232
